In [1]:
import ovito
from ovito.io import import_file
from ovito.modifiers import *
from ovito.data import *
import glob as glob
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import tarfile

from clusterOVITO.basic.io import *
from clusterOVITO.basic.xyz_to_FEFF import *
from clusterOVITO.ovito_tools import *  
from ovito.modifiers import ReplicateModifier


from clusterOVITO.basic.xyz_to_FEFF import *

rmeshPrime = np.arange(0.01, 6, 0.01)

In [15]:
class MD_EXAFS_Binary_Pipeline:
    def __init__(self, pipeline, framerange, file_params):
        self.pipeline = pipeline
        self.framerange = framerange
        self.file_params = file_params
        #self.proj_name = file_params["proj_name"]
        #self.working_dir = file_params["working_dir"]
        #self.cluster_dir = file_params["cluster_dir"]
        #self.proj_dir = self.working_dir + self.proj_name + '/'
        self.frame = 0
        self.interesting_c = 0
        self.interesting_pt = 0
        self.all_pt = 0
        self.interesting_o = 0
        self.b_line_table = []

    def run(self):
        for frame in self.framerange:
            self.frame = frame
            data = self.pipeline.compute(frame)
            finder = CutoffNeighborFinder(3, data)

            ptatom_index, catom_index, oatom_index = self.get_atom_indices(data)
            all_positions = data.particles.positions[...]

            interesting_c, interesting_pt, interesting_o = self.find_interesting_atoms(
                catom_index, ptatom_index, oatom_index, finder
            )

            self.interesting_c = all_positions[interesting_c]
            self.interesting_pt = all_positions[interesting_pt]
            self.interesting_o = all_positions[interesting_o]
            self.all_pt = all_positions[ptatom_index]

            self.add_sep_line_start()
            self.make_files()
        self.finish()


    def get_atom_indices(self, data):
        ptatom_index = np.where(data.particles['Particle Type'].array == 1)[0]
        catom_index = np.where(data.particles['Particle Type'].array == 2)[0]
        oatom_index = np.where(data.particles['Particle Type'].array == 3)[0]
        return ptatom_index, catom_index, oatom_index

    def find_interesting_atoms(self, catom_index, ptatom_index, oatom_index, finder):
        interesting_c, interesting_pt, interesting_o = [], [], []
        for i in catom_index:
            neigh = [neigh.index for neigh in finder.find(i)]
            if len(neigh) > 0:
                n_count = 0
                for l in neigh:
                    if l in ptatom_index:
                        n_count += 1
                        interesting_pt.append(l)
                if n_count > 0:
                    interesting_c.append(i)

        interesting_c = np.unique(np.array(interesting_c))
        interesting_pt = np.unique(np.array(interesting_pt))

        for i in interesting_c:
            neigh = [neigh.index for neigh in finder.find(i)]
            if len(neigh) > 0:
                for l in neigh:
                    if l in oatom_index:
                        interesting_o.append(l)

        interesting_o = np.unique(np.array(interesting_o))

        return interesting_c, interesting_pt, interesting_o
    
    def make_files(self):
        n_c = self.interesting_c.shape[0]
        n_pt = self.all_pt.shape[0]
        n_o = self.interesting_o.shape[0]

        n_pt_pot = 1
        n_c_pot = 2
        n_o_pot = 3

        n_pt_pots = np.full(n_pt, 1).reshape(-1, 1)
        pt_lattice = np.concatenate((self.all_pt, n_pt_pots), axis=1)
        
        n_c_pots = np.full(n_c, 2).reshape(-1, 1)
        c_lattice = np.concatenate((self.interesting_c, n_c_pots), axis=1)

        n_o_pots = np.full(n_o, 3).reshape(-1, 1)
        o_lattice = np.concatenate((self.interesting_o, n_o_pots), axis=1)
        

        b_line = np.concatenate((pt_lattice, c_lattice, o_lattice), axis=0)
        b_line = np.around(b_line, decimals=4)
        self.b_line_table.append(b_line)


    def add_sep_line_start(self):
        self.b_line_table.append(np.array([[0,0,0,0]]))

    def add_sep_line_end(self):
        self.b_line_table.append(np.array([1,1,1,1]))

    def finish(self):
        self.data = np.concatenate(self.b_line_table, axis=0)


In [3]:
###
# Make input table and export
###

import struct
import io
import tarfile

###########################################

traj_dir = '/mnt/a/MD_Trajectories/PtCO/NPs/'
traj_name = "Pt309_cuboct_298K.all.bin"

p_name = "Pt309_cuboct_298K"
#input_dir = "/mnt/a/MD_Trajectories/PtCO/NPs/Inputs/"
input_dir = "/mnt/sdcc/sdcc+u/nmarcella/MD_EXAFS_inputs/231228/"


###########################################
traj_path = traj_dir + traj_name

dir_name = input_dir + p_name + "/"

# if it doesn't exist, make it
os.makedirs(dir_name, exist_ok=True)

file_name = p_name + ".bin"
tar_name = file_name + ".tar.gz"


pipeline = import_file(traj_path)

test_pl = MD_EXAFS_Binary_Pipeline(pipeline, range(15999,18000), file_params={})
test_pl.run()

data = test_pl.data

###########################################

# Create a BytesIO object to accumulate the binary data
binary_buffer = io.BytesIO()

# Assuming 'data' is a 2D array where each row is [x, y, z, p]
for line in data:
    # Extract x, y, z, and p from the line
    x, y, z, p = line[0], line[1], line[2], int(line[3])

    # Pack the data into a binary format and write to the BytesIO object
    binary_data = struct.pack("fffI", x, y, z, p)
    binary_buffer.write(binary_data)

# Rewind the buffer to the beginning
binary_buffer.seek(0)

# Create a .tar.gz file and add the BytesIO object
tar_file_name = dir_name + tar_name
with tarfile.open(tar_file_name, "w:gz") as tar:
    # Create a TarInfo object for the BytesIO object
    info = tarfile.TarInfo(name=file_name)
    info.size = len(binary_buffer.getbuffer())
    
    # Add the BytesIO object to the tar archive
    tar.addfile(tarinfo=info, fileobj=binary_buffer)


In [17]:
pipeline = import_file(traj_path)

test_pl = MD_EXAFS_Binary_Pipeline(pipeline, range(15999,16000), file_params={})
test_pl.run()

data = test_pl.data

In [18]:
data.shape

(446, 4)

In [2]:
test = {"Pt":1, "C":2, "O":3}

In [22]:
class MD_EXAFS_Binary_Pipeline_General:
    def __init__(self, pipeline, file_params, custom_analysis_func=None):
        self.pipeline = pipeline
        self.framerange = file_params["framerange"]
        self.atoms = file_params["atoms"]
        self.cutoff = file_params["cutoff"]
        self.custom_analysis_func = custom_analysis_func
        self.b_line_table = []

    def run(self):
        for frame in self.framerange:
            self.frame = frame
            data = self.pipeline.compute(frame)
            finder = CutoffNeighborFinder(self.cutoff, data)

            atomic_position_index_list = self.get_atom_indices(data)
            all_positions = data.particles.positions[...]

            if self.custom_analysis_func:
            
                custom_result = self.custom_analysis_func(
                    atomic_position_index_list, finder
                    )
                self.interesting_results = []
                # add metal first
                self.interesting_results.append(all_positions[atomic_position_index_list[0]])
                # add other atoms
                for result in custom_result:
                    self.interesting_results.append(all_positions[result])
            # just add metal
            else:
                self.interesting_results = [all_positions[atomic_position_index_list[0]]]

            self.add_sep_line_start()

            self.make_files()

        self.finish()

    def get_atom_indices(self, data):

        atomic_position_index_list = []
        for atom in self.atoms.values():
            atomic_position_index_list.append(np.where(data.particles['Particle Type'].array == atom)[0])

        return atomic_position_index_list
    

    def make_lattice(self, n_atoms, n_pot, atom_coords):
            n_pots = np.full(n_atoms, n_pot).reshape(-1, 1)
            lattice = np.concatenate((atom_coords, n_pots), axis=1)
            return lattice

    def make_files(self):
        
        n_atoms = [atom.shape[0] for atom in self.interesting_results]
        
        atom_pots = [pot for pot in self.atoms.values()]

        lattices = [self.make_lattice(n_atom, n_pot, atom_coords) for n_atom, n_pot, atom_coords in zip(n_atoms, atom_pots, self.interesting_results)]

        b_line = np.concatenate(lattices, axis=0)
        b_line = np.around(b_line, decimals=4)
        self.b_line_table.append(b_line)


    def add_sep_line_start(self):
        self.b_line_table.append(np.array([[0,0,0,0]]))

    def add_sep_line_end(self):
        self.b_line_table.append(np.array([1,1,1,1]))

    def finish(self):
        self.data = np.concatenate(self.b_line_table, axis=0)


def find_adsorbed_CO(atomic_position_index_list, finder):
        ptatom_index, catom_index, oatom_index = atomic_position_index_list
        interesting_c, interesting_pt, interesting_o = [], [], []
        for i in catom_index:
            neigh = [neigh.index for neigh in finder.find(i)]
            if len(neigh) > 0:
                n_count = 0
                for l in neigh:
                    if l in ptatom_index:
                        n_count += 1
                        interesting_pt.append(l)
                if n_count > 0:
                    interesting_c.append(i)

        interesting_c = np.unique(np.array(interesting_c))
        interesting_pt = np.unique(np.array(interesting_pt))

        for i in interesting_c:
            neigh = [neigh.index for neigh in finder.find(i)]
            if len(neigh) > 0:
                for l in neigh:
                    if l in oatom_index:
                        interesting_o.append(l)

        interesting_o = np.unique(np.array(interesting_o))

        return interesting_c, interesting_o

In [36]:


###########################################

traj_dir = '/mnt/a/MD_Trajectories/PtCO/NPs/'
traj_name = "Pt309_cuboct_298K.all.bin"

p_name = "Pt309_cuboct_298K"
#input_dir = "/mnt/a/MD_Trajectories/PtCO/NPs/Inputs/"
input_dir = "/mnt/sdcc/sdcc+u/nmarcella/MD_EXAFS_inputs/231228/"


###########################################
traj_path = traj_dir + traj_name

pipeline = import_file(traj_path)

#test_pl = MD_EXAFS_Binary_Pipeline_General(pipeline, file_params={"framerange": range(15999,16000),"atoms":{"Pt":1, "C":2, "O":3}, "cutoff":3}, custom_analysis_func=find_adsorbed_CO)
test_pl = MD_EXAFS_Binary_Pipeline_General(pipeline, file_params={"framerange": range(15999,16000),"atoms":{"Pt":1}, "cutoff":3})
test_pl.run()

data = test_pl.data

In [42]:
###
# Make input table and export
###

import struct
import io
import tarfile

###########################################

traj_dir = '/mnt/a/MD_Trajectories/PtCO/NPs/'
traj_name = "Pt309_cuboct_298K.all.bin"

p_name = "Pt309_cuboct_298K_noCO"
#input_dir = "/mnt/a/MD_Trajectories/PtCO/NPs/Inputs/"
input_dir = "/mnt/sdcc/sdcc+u/nmarcella/MD_EXAFS_inputs/231228/"


###########################################
traj_path = traj_dir + traj_name

dir_name = input_dir + p_name + "/"

# if it doesn't exist, make it
os.makedirs(dir_name, exist_ok=True)

file_name = p_name + ".bin"
tar_name = file_name + ".tar.gz"


pipeline = import_file(traj_path)

test_pl = MD_EXAFS_Binary_Pipeline_General(pipeline, file_params={"framerange": range(15999,18000),"atoms":{"Pt":1}, "cutoff":3})
test_pl.run()

data = test_pl.data

###########################################

# Create a BytesIO object to accumulate the binary data
binary_buffer = io.BytesIO()

# Assuming 'data' is a 2D array where each row is [x, y, z, p]
for line in data:
    # Extract x, y, z, and p from the line
    x, y, z, p = line[0], line[1], line[2], int(line[3])

    # Pack the data into a binary format and write to the BytesIO object
    binary_data = struct.pack("fffI", x, y, z, p)
    binary_buffer.write(binary_data)

# Rewind the buffer to the beginning
binary_buffer.seek(0)

# Create a .tar.gz file and add the BytesIO object
tar_file_name = dir_name + tar_name
with tarfile.open(tar_file_name, "w:gz") as tar:
    # Create a TarInfo object for the BytesIO object
    info = tarfile.TarInfo(name=file_name)
    info.size = len(binary_buffer.getbuffer())
    
    # Add the BytesIO object to the tar archive
    tar.addfile(tarinfo=info, fileobj=binary_buffer)


In [43]:
dir_name

'/mnt/sdcc/sdcc+u/nmarcella/MD_EXAFS_inputs/231228/Pt309_cuboct_298K_noCO/'